In [3]:
import os
import pandas as pd
import asyncio
import time
from openai import OpenAI
from dotenv import load_dotenv

In [2]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY not found")

client = OpenAI(api_key=api_key)

In [13]:
df = pd.read_csv("final_data_sampled.csv")
df

,Unnamed: 0,rating,text,business_name,business_category,business_description,_id
0,848717,5,great for sunset,'Ehukai Beach Park,"['Park', 'Public beach', 'Tourist attraction']",Popular surfing beach offering massive wintert...,1.1426748616291053e+20_1589655333572
1,848703,5,The main access point to popular surfing break...,'Ehukai Beach Park,"['Park', 'Public beach', 'Tourist attraction']",Popular surfing beach offering massive wintert...,1.1447491719186307e+20_1591852064598
2,848713,5,Great place to set up your camping chairs and ...,'Ehukai Beach Park,"['Park', 'Public beach', 'Tourist attraction']",Popular surfing beach offering massive wintert...,1.0893375045945795e+20_1463461472578
3,848706,5,My favorite Beach for surfing on Oahu North Sh...,'Ehukai Beach Park,"['Park', 'Public beach', 'Tourist attraction']",Popular surfing beach offering massive wintert...,1.1249899958787118e+20_1570685676722
4,848694,5,Excellent beach for family activities great su...,'Ehukai Beach Park,"['Park', 'Public beach', 'Tourist attraction']",Popular surfing beach offering massive wintert...,1.1730942640485394e+20_1605375558437
...,...,...,...,...,...,...,...
9995,1335348,5,"Very delicious, and a great value for money.\n...",Hawaiian Style Cafe Hilo,"['Hawaiian restaurant', 'Cafe', 'Restaurant']",Classic Hawaiian comfort dishes are served at ...,1.1128541514815295e+20_1551226917271
9996,1258876,2,Not sweet look like water,Starbucks,"['Coffee shop', 'Breakfast restaurant', 'Cafe'...",Seattle-based coffeehouse chain known for its ...,1.152926481959341e+20_1577077386910
9997,55633,5,"Great service, great food. Well worth the wait...",The Cheesecake Factory,"['Restaurant', 'American restaurant', 'Dessert...",American chain restaurant offering sizable por...,1.0790129418280239e+20_1561055372429
9998,86013,4,Great falls to visit and kids enjoyed it. It'...,Akaka Falls State Park,"['State park', 'Tourist attraction']",Tropical rainforest park offering a .4-mile lo...,1.0844578957930978e+20_1490221161262


In [5]:
with open("policy.md", "r", encoding="utf-8") as f:
    policy_text = f.read()

In [42]:
row = df.iloc[300]
print(row["text"])
print(row["rating"])
print(row["business_name"])
print(row["business_category"])
print(row["business_description"])

As expected
4
Anuenue Cafe
['Breakfast restaurant', 'Brunch restaurant', 'Coffee shop', 'Coffee store', 'Gluten-free restaurant', 'Hawaiian restaurant', 'Health food restaurant', 'Sandwich shop', 'Vegetarian restaurant']
Cozy local eatery whipping up hearty American breakfast chow in snug quarters with outdoor seating.


In [45]:
def make_prompt(row):
    row_text = "\n".join([f"{col}: {row[col]}" for col in df.columns])
    return f"""
            {policy_text}

            Row data:
            {row_text}

            Respond only with 0 (not trustworthy) or 1 (trustworthy).
            """

# test for 1 line
prompt = make_prompt(row)

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}],
    temperature=0
)

label = response.choices[0].message.content.strip()
label = "1" if "1" in label else "0"

print("Predicted label:", label)

Predicted label: 0


In [44]:
print(row["text"])
print(row["rating"])
print(row["business_name"])
print(row["business_category"])
print(row["business_description"])

As expected
4
Anuenue Cafe
['Breakfast restaurant', 'Brunch restaurant', 'Coffee shop', 'Coffee store', 'Gluten-free restaurant', 'Hawaiian restaurant', 'Health food restaurant', 'Sandwich shop', 'Vegetarian restaurant']
Cozy local eatery whipping up hearty American breakfast chow in snug quarters with outdoor seating.
